# Create Zarr Stores with Different Chunk Shapes

In this notebook, we create Zarr stores for the CMIP6 TAS daily data available in NetCDF on S3. This method of creating Zarr stores uses [pangeo-forge](https://pangeo-forge.readthedocs.io/) and it's [recipes](https://pangeo-forge.readthedocs.io/en/latest/pangeo_forge_recipes/recipe_user_guide/index.html) pattern.

## 1.1 Install and import libraries

In [1]:
%%capture
!pip uninstall apache-beam -y
!pip install 'apache-beam[interactive, dataframe]==2.48.0' git+https://github.com/carbonplan/cmip6-downscaling.git git+https://github.com/pangeo-forge/pangeo-forge-recipes.git@beam-refactor

In [2]:
import apache_beam as beam
import boto3
from botocore.exceptions import ClientError
import fsspec
import os
from pangeo_forge_recipes.patterns import FilePattern, ConcatDim, MergeDim
from pangeo_forge_recipes.transforms import OpenURLWithFSSpec, OpenWithXarray, StoreToZarr
from pangeo_forge_recipes.storage import FSSpecTarget
import re
import rioxarray
import s3fs
import xarray as xr
import eodc_hub_role

In [3]:
credentials = eodc_hub_role.fetch_and_set_credentials()
bucket = 'nasa-eodc-data-store'

Note: This is adapted from https://github.com/carbonplan/benchmark-maps/blob/datasets/stores/01b_cmip6_netcdf_to_zarr.ipynb.

## 1.2 Set parameters

In [4]:
#parameters
temporal_resolution = "daily"
model = "GISS-E2-1-G"
variable = "tas"
anon=True

In [5]:
# Initiate fsspec filesystems for reading and writing
s3_path = f"s3://nex-gddp-cmip6/NEX-GDDP-CMIP6/{model}/historical/r1i1p1*/{variable}/*"
fs_read = fsspec.filesystem("s3", anon=anon, skip_instance_cache=False)
fs_write = fsspec.filesystem("")

In [6]:
# Retrieve list of available months
files_paths = fs_read.glob(s3_path)
print(f"{len(files_paths)} discovered from {s3_path}")

65 discovered from s3://nex-gddp-cmip6/NEX-GDDP-CMIP6/GISS-E2-1-G/historical/r1i1p1*/tas/*


In [7]:
files_paths[0]

'nex-gddp-cmip6/NEX-GDDP-CMIP6/GISS-E2-1-G/historical/r1i1p1f2/tas/tas_day_GISS-E2-1-G_historical_r1i1p1f2_gn_1950.nc'

## 1.3 Test we can open the files

In [8]:
fs_s3 = s3fs.S3FileSystem(anon=True)
filepath = f's3://{files_paths[0]}'
f = fs_s3.open(filepath, mode='rb')
ds = xr.open_dataset(f)
ds

<xarray.Dataset>
Dimensions:  (time: 365, lat: 600, lon: 1440)
Coordinates:
  * time     (time) object 1950-01-01 12:00:00 ... 1950-12-31 12:00:00
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
Data variables:
    tas      (time, lat, lon) float32 ...
Attributes: (12/23)
    downscalingModel:      BCSD
    activity:              NEX-GDDP-CMIP6
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    Conventions:           CF-1.7
    creation_date:         2021-10-04T18:41:40.796912+00:00
    frequency:             day
    ...                    ...
    history:               2021-10-04T18:41:40.796912+00:00: install global a...
    disclaimer:            This data is considered provisional and subject to...
    external_variables:    areacella
    cmip6_source_id:       GISS-E2-1-G
    cmip6_institution_id:  NASA-GISS
    cmip6_license:         CC-BY-SA 4.0

# 2: Setup the destination

In [12]:
def format_function(time):
    pattern = r"\b\d{4}\b"
    return re.sub(pattern, str(time), filepath)

years = list(range(1950, 1952))
time_dim = ConcatDim("time", keys=years)

pattern = FilePattern(format_function, time_dim, file_type="netcdf4")
pattern = FilePattern.prune(pattern, nkeep=2)

In [13]:
pattern.combine_dims

(ConcatDim(name='time', nitems_per_file=1),)

In [14]:
fs = s3fs.S3FileSystem(
    key=credentials['AccessKeyId'],
    secret=credentials['SecretAccessKey'],
    token=credentials['SessionToken'], 
    anon=False
)
target_root = FSSpecTarget(fs=fs, root_path=bucket)
print(f"Using {pattern.items()}")

Using <generator object FilePattern.items at 0x7fa1c73efe60>


# 3: Set different target chunks

For different sets of chunks, generate a zarr store.

In [15]:
chunk_sets = []
# Optimized for analysis
temporal_target_chunks = { 'lat': ds.lat.shape[0], 'lon': ds.lon.shape[0], 'time': 29 }
chunk_sets.append(temporal_target_chunks)

In [12]:
# Optimized for visualization at a single time step
global_target_chunks = { 'lat': ds.lat.shape[0], 'lon': ds.lon.shape[0], 'time': 1 }
global_target_chunks
chunk_sets.append(global_target_chunks)

In [13]:
# Optimized for time series
#spatial_target_chunks = calc_auspicious_chunks_dict(ds[variable], chunk_dims=('lat','lon',))
spatial_target_chunks = {'time': 365, 'lat': 262, 'lon': 262}
chunk_sets.append(spatial_target_chunks)

In [14]:
chunk_sets

[{'lat': 600, 'lon': 1440, 'time': 29},
 {'lat': 600, 'lon': 1440, 'time': 1},
 {'time': 365, 'lat': 262, 'lon': 262}]

In [16]:
s3 = boto3.client(
    's3',
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken']    
)

for chunk_set in chunk_sets:
    dir_path = str(("_").join(map(str, chunk_set.values()))) + "_2"
    store_name = f"{dir_path}/CMIP6_{temporal_resolution}_{model}_{variable}.zarr"
    try:
        key = f"{store_name}/.zmetadata"
        response = s3.head_object(Bucket=bucket, Key=key)
        print(f"File '{store_name}' exists in bucket '{bucket}'.")
        continue
    except ClientError as e:
        if e.response['Error']['Code'] == '404':
            print(f"File '{key}' does not exist in bucket '{bucket}'.")
        else:
            print(f"Error occurred: {e}")
            raise e
    print(f"Writing to {target_root}/{store_name}")
    transforms = (
        beam.Create(pattern.items())
        | OpenURLWithFSSpec(open_kwargs={'anon': True})
        | OpenWithXarray(file_type=pattern.file_type)
        | StoreToZarr(
            store_name=store_name,
            target_root=target_root,
            combine_dims=pattern.combine_dim_keys,
            target_chunks=chunk_set,
        )
    )
    # Commented out so we don't re-run if we don't intend to
    with beam.Pipeline() as p:
        p | transforms

File '600_1440_29_2/CMIP6_daily_GISS-E2-1-G_tas.zarr/.zmetadata' does not exist in bucket 'nasa-eodc-data-store'.
Writing to FSSpecTarget(fs=<s3fs.core.S3FileSystem object at 0x7fa1bde56bc0>, root_path='nasa-eodc-data-store')/600_1440_29_2/CMIP6_daily_GISS-E2-1-G_tas.zarr


In [19]:
!aws s3 ls s3://nasa-eodc-data-store/600_1440_29_2/CMIP6_daily_GISS-E2-1-G_tas.zarr/time/

2023-07-29 15:57:13        316 .zarray
2023-07-29 15:57:13        187 .zattrs
2023-07-29 15:57:18         99 0
2023-07-29 15:57:21         81 1
2023-07-29 15:57:45         99 10
2023-07-29 15:57:48         81 11
2023-07-29 15:57:50        101 12
2023-07-29 15:57:52         81 13
2023-07-29 15:57:53         99 14
2023-07-29 15:57:55         81 15
2023-07-29 15:57:57         86 16
2023-07-29 15:57:59         99 17
2023-07-29 15:58:00         81 18
2023-07-29 15:58:02         99 19
2023-07-29 15:57:23         99 2
2023-07-29 15:58:04         81 20
2023-07-29 15:58:05         81 21
2023-07-29 15:58:07         99 22
2023-07-29 15:58:08         81 23
2023-07-29 15:58:10         86 24
2023-07-29 15:58:11         76 25
2023-07-29 15:57:26         81 3
2023-07-29 15:57:28         83 4
2023-07-29 15:57:32         99 5
2023-07-29 15:57:34         81 6
2023-07-29 15:57:37         86 7
2023-07-29 15:57:39         99 8
2023-07-29 15:57:43         81 9


In [16]:
dir_path = str(("_").join(map(str, chunk_sets[1].values())))
store_name = f"{dir_path}/CMIP6_{temporal_resolution}_{model}_{variable}.zarr"

In [17]:
store = s3fs.S3Map(root=f"{bucket}/{store_name}", s3=fs, check=True)

In [18]:
xr.open_zarr(store, consolidated=True)

<xarray.Dataset>
Dimensions:  (lat: 600, lon: 1440, time: 730)
Coordinates:
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) object 1950-01-01 12:00:00 ... 1950-12-31 12:00:00
Data variables:
    tas      (time, lat, lon) float32 dask.array<chunksize=(1, 600, 1440), meta=np.ndarray>
Attributes: (12/23)
    Conventions:           CF-1.7
    activity:              NEX-GDDP-CMIP6
    cmip6_institution_id:  NASA-GISS
    cmip6_license:         CC-BY-SA 4.0
    cmip6_source_id:       GISS-E2-1-G
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    ...                    ...
    scenario:              historical
    source:                BCSD
    title:                 GISS-E2-1-G, r1i1p1f2, historical, global downscal...
    tracking_id:           25d6baa3-0404-4eba-a3f1-afddbf69d4cc
    variant_label:         r1i1p1f2
    version:               1.0

In [19]:
store_name

'600_1440_1/CMIP6_daily_GISS-E2-1-G_tas.zarr'

# 4: Check it worked

In [20]:
for chunk_set in chunk_sets:
    dir_path = str(("_").join(map(str, chunk_set.values())))
    store_name = f"{dir_path}/CMIP6_{temporal_resolution}_{model}_{variable}.zarr"
    key = f"{store_name}/.zmetadata"
    response = s3.head_object(Bucket=bucket, Key=key)
    object_size = response['ContentLength']
    object_size_MB = object_size / (1024)    
    print(f"Size of metadata {object_size_MB} KB")
    
    store = s3fs.S3Map(root=f"{bucket}/{store_name}", s3=fs, check=True)
    ds = xr.open_zarr(store, consolidated=True)
    print(ds)

Size of metadata 5.2890625 KB
<xarray.Dataset>
Dimensions:  (lat: 600, lon: 1440, time: 730)
Coordinates:
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) object 1950-01-01 12:00:00 ... 1950-12-31 12:00:00
Data variables:
    tas      (time, lat, lon) float32 dask.array<chunksize=(29, 600, 1440), meta=np.ndarray>
Attributes: (12/23)
    Conventions:           CF-1.7
    activity:              NEX-GDDP-CMIP6
    cmip6_institution_id:  NASA-GISS
    cmip6_license:         CC-BY-SA 4.0
    cmip6_source_id:       GISS-E2-1-G
    contact:               Dr. Rama Nemani: rama.nemani@nasa.gov, Dr. Bridget...
    ...                    ...
    scenario:              historical
    source:                BCSD
    title:                 GISS-E2-1-G, r1i1p1f2, historical, global downscal...
    tracking_id:           25d6baa3-0404-4eba-a3f1-afddbf69d4cc
    variant_label: 